In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles
using Random

In [2]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [3]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity30_.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance30.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"),header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File(string(path,"D_Dsoc.csv"),header=1,delim=",") |> DataFrame
emi_sink = CSV.File(string(path,"emission_sink.csv"),header=1,delim=",") |> DataFrame;

In [4]:
# distance[distance.Column1 .∈ countries, :]

In [5]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi

emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"] .* 0.6          # national CO2 sink (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
# EF_trans = 86/1000000                          #5-LH trucks emit on average 56.6 gCO2/tkm while 5-RD emit 84.0 gCO2/tkm
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
penalty = 5; # ton/yr;

In [6]:
# # scen_eff = CSV.File(string(path,"Scenarios/FeaEff_15.csv"), header=1, delim=",") |> DataFrame
# scen_eff = CSV.File(string(path,"Scenarios/Scen_Efficiency2.csv"), header=1, delim=",") |> DataFrame
# scen_eff = Matrix(scen_eff)[:, 2:end]

# # scen_d = CSV.File(string(path,"Scenarios/FeaDmd_15.csv"), header=1, delim=",") |> DataFrame
# # scen_dmd = Matrix(scen_d).*1.2

# scen_dmd = ones(16) .* 452000

# num_omega = size(scen_eff)[2]
# Omega = ones(Float64, ncty, num_omega, nproc);  # Omega[ncty,nscena,nproc]
# Omega[:, :, 1] = scen_eff;

# dmd_fixed = 452000
# # for k in 1:nproc
# #     Omega[:, :, k] = scen_eff;
# # end

In [7]:
# list = [1,4,5,31,37,38,49];
list = [1,4,5,11,21,31,37,41,49];

In [8]:
scen_eff = CSV.File(string(path,"Scenarios/Scen_Efficiency2.csv"), header=1, delim=",") |> DataFrame
scen_eff = Matrix(scen_eff)[:, 2:end]
scen_eff = scen_eff[:, list]
num_omega = size(scen_eff)[2]
scen_dmd = ones(num_omega) .* 452000

Omega = ones(Float64, ncty, num_omega, nproc);  # Omega[ncty,nscena,nproc]
Omega[:, :, 1] = scen_eff;

dmd_fixed = 452000

452000

---

### Solve Subproblem - Get Dual $\pi$ & $\alpha$

In [9]:
function subprob(x_hat, s) # s represents specific scenario     
    
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, unmeet >= 0)

    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty] 
    
    cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]
    cstr_fmko = [@constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0) for j in filter!(e->e!=mkt_loc,collect(1:ncty))]
    
    
    transD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
        end
        transD[j] = arc_emi  # ncty*1 matrix
    end
    
    
    ############# over produced part wont get supply ###########
    sink_ratio = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            sink_ratio[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    overprod_penalty = sum((x_hat - sum(y, dims=2)[:,1,:]) .* sink_ratio)
    
    
    ############### Obj function ##############
    @objective(model, Min, sum(transD) + unmeet*penalty + overprod_penalty)
    JuMP.optimize!(model)
    
    qy_hat = JuMP.objective_value(model)
    sub_y = JuMP.value.(y)
    sub_unmeet = JuMP.value.(unmeet)
    
    
    ################## get dual and calculate cuts ################
    dual_pi = zeros(ncty,nproc)
    
    op_pi = [getdual(con) for con in cstr_op]
    cth_pi = [getdual(con) for con in cstr_cth]
    cell_pi = [getdual(con) for con in cstr_cell]
    noncell_pi = [getdual(con) for con in cstr_noncell]
    battery_pi = [getdual(con) for con in cstr_battery]
    fmk_alp = [getdual(con) for con in cstr_fmk]
    fmko_alp = [getdual(con) for con in cstr_fmko]
    
    dual_pi[:,:] = reshape(op_pi, ncty, nproc) .* Omega[:,s,:]
    dual_pi[:, cathode] = reshape(cth_pi, ncty, length(cathode)) .* Omega[:,s,cathode]
    dual_pi[:, cell] = reshape(cell_pi, ncty, length(cell)) .* Omega[:,s,cell]
    dual_pi[:, noncell] = reshape(noncell_pi, ncty, length(noncell)) .* Omega[:,s,noncell]
    dual_pi[:, battery] = reshape(battery_pi, ncty, length(battery)) .* Omega[:,s,battery]
    dual_alpha = sum(fmk_alp*(scen_dmd[s] - sub_unmeet)) + sum(fmko_alp * 0)

    #######################
    result = Dict(["qyhat"=>qy_hat, "y_opt"=>sub_y, "unmeet_opt"=>sub_unmeet, "pi"=>dual_pi, "alp"=>dual_alpha]);
    return result
    
end

subprob (generic function with 1 method)

In [10]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= 0.8*capacity[!, 2:end][i,k])
        end
    end

    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];
#     x_demand = @constraint(model, sum(x[i,15] for i in 1:ncty) == dmd_fixed) 
    

    ###########
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    
    G = 0
    g = 0
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, theta >= sum(x .* G) + g); 
        end
    end
    
    
    @objective(model, Min, sum(proD-proS)+theta)
    JuMP.optimize!(model);
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result
end

masterprob (generic function with 1 method)

In [11]:
function add_cuts(x)
    G = [] # gradient
    g = [] # intersection
    qy = 0
    
    for s in 1:num_omega
        res = subprob(x, s)        
        push!(G, res["pi"])   # gradient
        push!(g, res["alp"])  # intersection
        qy += res["qyhat"]
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    
    z_hat = sum(proD - proS) + qy/num_omega
    Exp_G = sum(G) / num_omega
    Exp_g = sum(g) / num_omega
    
    cut = Dict(["gradient"=>Exp_G, "intersection"=>Exp_g])
    return cut, z_hat
end

add_cuts (generic function with 1 method)

In [12]:
function main(toler)
    i = 0
    z_ub = 1e8
    x_opt = 0
    
    cuts = []
    res0 = masterprob(cuts)
    x_hat = res0["x_hat"]
    z_lb = res0["z_lb"]
    
    while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
        new_cut, z_hat = add_cuts(x_hat)
        
        if z_hat < z_ub
            z_ub = z_hat
            x_opt = x_hat
        end
        
        push!(cuts, new_cut)
        
        res1 = masterprob(cuts)
        z_lb = res1["z_lb"]
        x_hat = res1["x_hat"]
        
        i += 1
    end
    
    result = Dict(["z_lb"=>z_lb, "z_ub"=>z_ub, "x_opt"=>x_opt, "iteration"=>i])
    return result
    
end

main (generic function with 1 method)

-----

In [13]:
M = -100000
toler = 0.01

0.01

In [14]:
i = 0
z_ub = 1e8
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

Academic license - for non-commercial use only - expires 2024-12-26


-3.4870410868231794e6

In [15]:
x_hat

30×15 Matrix{Float64}:
  4960.0       0.0  0.0           0.0  …     0.0   0.0        0.0
 46959.8       0.0  0.0           0.0        0.0   0.0        0.0
  1760.0       0.0  1.1623e5  53600.0        0.0   0.0        0.0
   400.0    2480.0  0.0       65524.5        0.0   0.0        0.0
 31200.0       0.0  0.0           0.0        0.0   7.75558e5  0.0
     0.0       0.0  0.0           0.0  …     0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0  116000.0  0.0           0.0        0.0   0.0        0.0
     0.0     867.2  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0  …  3102.23  0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     ⋮                                 ⋱                      
     0.0       0.0  0.0           0.0        0.0   0.0  

In [295]:
z_ub - z_lb, toler * min(abs(z_ub), abs(z_lb))

(1.0348704108682318e8, 34870.4108682318)

In [296]:
new_cut, z_hat = add_cuts(x_hat)

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26


(Dict{String, Any}("gradient" => [-1.3220747403589865 -1.2104112555449973 … -0.9255104467762373 -1.2044061861658102; -2.198557475293465 -1.4560881958799996 … -0.14326094365592024 -0.5022742452579547; … ; -0.7804945007042557 -0.48022544401350054 … -0.17495540134682266 0.0; -1.0107935004455875 -0.7948778024324987 … -0.14348608036663746 0.0], "intersection" => -1.6224751274787537e6), -557701.6138260942)

In [297]:
new_cut["gradient"]

30×15 Matrix{Float64}:
 -1.32207   -1.21041   -4.85117  …  -12.7139  -0.92551     -1.20441
 -2.19856   -1.45609   -3.59755     -13.9676  -0.143261    -0.502274
 -0.674395  -0.918405  -4.6173      -12.9479  -0.6916      -5.14866
 -0.11092   -0.64476   -4.34365     -13.7312  -0.311251    -5.04571
 -1.65437   -1.22175   -4.92064     -12.6445  -0.994974   -35.8292
 -0.999412  -0.739851  -3.44476  …  -14.0936  -0.067607     0.0
 -1.27727   -0.923744  -4.52142     -13.0437  -0.595756    -3.38083
 -0.969931  -0.616408  -3.88279     -13.6823  -0.619772     0.0
 -0.517268  -0.163745  -3.77674     -13.9973  -0.340847    -2.92369
 -0.434541  -0.081017  -3.77897     -13.7862  -0.203646     0.0
 -0.353524   0.0       -3.69889  …  -13.8662  -0.263        0.0
 -1.00623   -0.66288   -3.26439     -14.022   -0.0645703    0.0
 -1.50208   -1.14911   -3.30333     -14.2618  -0.108758     0.0
  ⋮                              ⋱                        
 -0.503747  -0.150223  -3.84911     -13.9327  -0.413224   

In [250]:
if z_hat < z_ub
    z_ub = z_hat
    x_opt = x_hat;
end

30×15 Matrix{Float64}:
  4960.0       0.0  0.0           0.0  …     0.0   0.0        0.0
 46959.8       0.0  0.0           0.0        0.0   0.0        0.0
  1760.0       0.0  1.1623e5  53600.0        0.0   0.0        0.0
   400.0    2480.0  0.0       65524.5        0.0   0.0        0.0
 31200.0       0.0  0.0           0.0        0.0   7.75558e5  0.0
     0.0       0.0  0.0           0.0  …     0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0  116000.0  0.0           0.0        0.0   0.0        0.0
     0.0     867.2  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0  …  3102.23  0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     ⋮                                 ⋱                      
     0.0       0.0  0.0           0.0        0.0   0.0  

In [252]:
push!(cuts, new_cut)

1-element Vector{Any}:
 Dict{String, Any}("gradient" => [-1.3220747403589865 -1.2104112555449973 … -0.9255104467762373 -1.2044061861658102; -2.198557475293465 -1.4560881958799996 … -0.14326094365592024 -0.5022742452579547; … ; -0.7804945007042557 -0.48022544401350054 … -0.17495540134682266 0.0; -1.0107935004455875 -0.7948778024324987 … -0.14348608036663746 0.0], "intersection" => -1.6224751274787537e6)

In [254]:
res1 = masterprob(cuts)

Academic license - for non-commercial use only - expires 2024-12-26


Dict{String, Any} with 2 entries:
  "x_hat" => [4960.0 0.0 … 0.0 0.0; 46959.8 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.…
  "z_lb"  => -1.3387e7

In [255]:
res1["x_hat"] == xhat1

true

In [256]:
model = Model(Gurobi.Optimizer)
@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, theta >= M)

for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= 0.8*capacity[!, 2:end][i,k])
    end
end

x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];


###########
proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1) 


# G = 0
# g = 0
# if cuts != []
#     for cut in cuts
#         G = cut["gradient"]
#         g = cut["intersection"]
#         @constraint(model, theta >= sum(x .* G) + g); 
#     end
# end


@objective(model, Min, sum(proD-proS)+theta)
JuMP.optimize!(model);

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 464 rows, 451 columns and 1290 nonzeros
Model fingerprint: 0xd147b2ef
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [3e-03, 2e+02]
  Bounds range     [1e+07, 1e+07]
  RHS range        [2e+02, 8e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 456 rows and 373 columns
Presolve time: 0.00s
Presolved: 8 rows, 78 columns, 137 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.7632856e+07   6.151405e+05   0.000000e+00      0s
      12   -1.3387041e+07   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.00 seconds
Optimal objective -1.338704109e+07

User-callback calls 56, time in user-callback 0.00 sec


In [259]:
model = Model(Gurobi.Optimizer)
@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, theta >= M)

for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= 0.8*capacity[!, 2:end][i,k])
    end
end

x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];


Academic license - for non-commercial use only - expires 2024-12-26


In [231]:
G = cuts[1]["gradient"]
g = cuts[1]["intersection"]
@constraint(model, theta >= sum(x .* G) + g)

1.3220747403589865 x[1,1] + 2.198557475293465 x[2,1] + 0.6743950229994301 x[3,1] + 0.11092038917519506 x[4,1] + 1.6543707363067766 x[5,1] + 0.9994117352487749 x[6,1] + 1.2772678395527557 x[7,1] + 0.9699313732067554 x[8,1] + 0.5172683792102557 x[9,1] + 0.43454075414605586 x[10,1] + 0.3535237239842557 x[11,1] + 1.0062281032532552 x[12,1] + 1.5020821858877555 x[13,1] + 0.7432841475692555 x[14,1] + 1.1255186035292555 x[15,1] + 0.7538218683407558 x[16,1] + 1.3286388690137556 x[17,1] + 1.3217247554387557 x[18,1] + 0.5037472008662558 x[19,1] + 1.3872129142742557 x[20,1] + 1.3989464510792557 x[21,1] + 0.4593266486874252 x[22,1] + 0.8492132917367555 x[23,1] + 0.7472937473267556 x[24,1] + 0.8959189903327027 x[25,1] + 1.128355851490756 x[26,1] + 0.5885971340372557 x[27,1] + 0.5032153786847556 x[28,1] + 0.7804945007042557 x[29,1] + 1.0107935004455875 x[30,1] + 1.2104112555449973 x[1,2] + 1.4560881958799996 x[2,2] + 0.9184051244235008 x[3,2] + 0.6447597785925012 x[4,2] + 1.2217468857149925 x[5,2] +

In [ ]:
###########
proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1) 

@objective(model, Min, sum(proD-proS)+theta)
JuMP.optimize!(model);

In [275]:
g

-1.6224751274787537e6

In [276]:
sum(-x_hat .* G)

8.983529970885588e6

In [264]:
-sum(x_hat .* G) + g

7.361054843406834e6

-----

In [185]:
function gen_fea(x_hat)
    ls = []
    for s in 1: num_omega
        model = Model(Gurobi.Optimizer)
        set_silent(model)
        @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
        @variable(model, unmeet >= 0)

        cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
        cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
        cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]
        cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
        cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == Omega[j,s,k] * x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty] 

        cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]
        cstr_fmko = [@constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0) for j in filter!(e->e!=mkt_loc,collect(1:ncty))]


        transD = Vector{AffExpr}(undef, ncty)
        for j in 1:ncty
            arc_emi = 0
            for i in 1:ncty
                amount = sum(y[i,j,k] for k in 1:nproc)
                arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
            end
            transD[j] = arc_emi  # ncty*1 matrix
        end

        sink_ratio = zeros(ncty, nproc)
        for k in 1:nproc
            for i in 1:ncty
                sink_ratio[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
            end
        end
        overprod_penalty = sum((x_hat - sum(y, dims=2)[:,1,:]) .* sink_ratio)



        @objective(model, Min, sum(transD) + unmeet*penalty + overprod_penalty)
        JuMP.optimize!(model)

        if termination_status(model) == MOI.OPTIMAL
            push!(ls, s)
        end
    end
    
    return ls
    
end

gen_fea (generic function with 1 method)